# 2. Entendimiento de los Datos (Exploratory Data Analysis - EDA)

Antes de proceder con cualquier técnica de modelado o feature engineering, es fundamental realizar analizar todos los campos del proyecto para tener una visual más amplia de los casos. En esta fase de **Entendimiento de los Datos**, buscamos auditar la calidad de la información, comprender la naturaleza de las variables y validar las hipótesis de negocio planteadas.

Nuestros objetivos principales en esta etapa son:

1.  **Calidad del dataset (Data Health Check):** Identificar valores nulos, registros duplicados, tipos de datos inconsistentes y cardinalidad excesiva que puedan afectar el entrenamiento.
2.  **Análisis de la Variable Objetivo (`claim_status`):** Confirmar la magnitud del desbalance de clases y establecer la línea base de siniestralidad.
3.  **Análisis Univariado y Bivariado:** Explorar la distribución de las variables numéricas y categóricas, y —más importante aún— visualizar su relación directa con la probabilidad de reclamo para detectar *drivers* de riesgo tempranos.

> **Nota:** Para mantener la integridad de la evaluación final, este análisis exploratorio y las decisiones de limpieza resultantes se basarán en el comportamiento general de los datos. Sin embargo, cualquier cálculo estadístico estricto (por ejemplo una imputación de datos) se ajustará posteriormente dentro de los *pipelines* de entrenamiento para evitar fugas de información (*data leakage*) del set de prueba.

In [ ]:
# importaciones
import re
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, train_test_split

In [2]:
# Visuals
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
warnings.filterwarnings("ignore")

In [3]:
# Routes
DATA_PATH = Path("../data")
RAW_DATA_FILE = DATA_PATH / "raw" / "claims_dataset.csv"
PROCESSED_DATA_PATH = DATA_PATH / "processed"
# Seed for reproducibility
SEED = 42

In [4]:
# Load dataset
try:
    df = pd.read_csv(RAW_DATA_FILE, sep=";")
    print(f"Dataset loaded successfully. Dimensions: {df.shape}")
except FileNotFoundError:
    print("Error! The file does not exist. Check the path.")

# Initial overview
df.head()

Dataset loaded successfully. Dimensions: (58592, 41)


,policy_id,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,POL045360,9.3,1.2,41,C8,8794,C2,M4,Diesel,250Nm@2750rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
1,POL016745,8.2,1.8,35,C2,27003,C1,M9,Diesel,200Nm@1750rpm,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,4,0
2,POL007194,9.5,0.2,44,C8,8794,C2,M4,Diesel,250Nm@2750rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
3,POL018146,5.2,0.4,44,C10,73430,A,M1,CNG,60Nm@3500rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
4,POL049011,10.1,1.0,56,C13,5410,B2,M5,Diesel,200Nm@3000rpm,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5,0


In [5]:
# Data Health Check Function
def data_health_check(df):
    print(f"1. Dimensiones del Dataset: {df.shape}")
    print("\n2. Duplicados:")
    print(f"   Filas duplicadas: {df.duplicated().sum()}")

    print("\n3. Tipos de Datos y Valores Nulos:")
    info_df = pd.DataFrame(df.dtypes, columns=["Dtype"])
    info_df["Nunique"] = df.nunique()
    info_df["Missing_Values"] = df.isnull().sum()
    info_df["%_Missing"] = (df.isnull().sum() / len(df)) * 100
    display(info_df.sort_values(by="%_Missing", ascending=False))


data_health_check(df)

1. Dimensiones del Dataset: (58592, 41)

2. Duplicados:
   Filas duplicadas: 0

3. Tipos de Datos y Valores Nulos:


,Dtype,Nunique,Missing_Values,%_Missing
policy_id,str,58592,0,0.0
subscription_length,float64,140,0,0.0
vehicle_age,float64,49,0,0.0
customer_age,int64,41,0,0.0
region_code,str,22,0,0.0
region_density,int64,22,0,0.0
segment,str,6,0,0.0
model,str,11,0,0.0
fuel_type,str,3,0,0.0
max_torque,str,9,0,0.0


In [6]:
# Creating a clean copy of the dataframe for further processing
df_clean = df.copy()

## 2.1 Limpieza y Preprocesamiento Inicial

Antes de iniciar el análisis exploratorio, se ejecutará una fase de limpieza de datos orientada a estructurar la información para el modelado. Las acciones específicas de limpieza incluyen:

* **Eliminación de Identificadores (`policy_id`):** Se descartará esta variable dado que posee una cardinalidad total (un valor único por registro), por lo que carece de valor predictivo. Su eliminación facilita además la detección real de registros duplicados basados en características del vehículo o cliente.
* **Estandarización Binaria:** Conversión de todas las variables categóricas dicotómicas (ej. "Yes"/"No", "True"/"False") a formato numérico entero (`1` / `0`). Esto habilita el cálculo directo de correlaciones y métricas de riesgo.
* **Extracción de Atributos de Motor:** Descomposición de las variables complejas `max_torque` y `max_power` (actualmente texto). Se extraerán dos nuevas características numéricas por variable: *Magnitud* (Fuerza/Potencia) y *Régimen* (RPM), permitiendo evaluar el impacto aislado de la potencia del vehículo en la siniestralidad.
* **Validación de Redundancia Regional:** Se realizará un test de unicidad entre `region_code` y `region_density`. Si se confirma una relación 1 a 1 (donde el código no aporta información adicional a la densidad), se eliminará `region_code` para evitar multicolinealidad. De lo contrario, se conservará para capturar riesgos geográficos latentes.
* **Saneamiento de Cadenas de Texto:** Aplicación de *trimming* (eliminación de espacios en blanco al inicio y final) en todas las variables categóricas para evitar la duplicación de categorías por errores de formato.
* **Deduplicación de Registros:** Tras la limpieza estructural (que puede generar filas idénticas al remover el ID), se procederá a eliminar los duplicados exactos. Esta acción es crítica para mitigar el riesgo de sobreajuste (*overfitting*) por memorización y prevenir la fuga de información (*data leakage*) entre los conjuntos de entrenamiento y prueba.

In [7]:
# Dropping an unnecessary column (if exists)
if "policy_id" in df_clean.columns:
    df_clean = df_clean.drop("policy_id", axis=1)
df_clean.head()

,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,9.3,1.2,41,C8,8794,C2,M4,Diesel,250Nm@2750rpm,113.45bhp@4000rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
1,8.2,1.8,35,C2,27003,C1,M9,Diesel,200Nm@1750rpm,97.89bhp@3600rpm,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,4,0
2,9.5,0.2,44,C8,8794,C2,M4,Diesel,250Nm@2750rpm,113.45bhp@4000rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
3,5.2,0.4,44,C10,73430,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
4,10.1,1.0,56,C13,5410,B2,M5,Diesel,200Nm@3000rpm,88.77bhp@4000rpm,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5,0


In [8]:
# Converting binary categorical variables to 0/1
binary_cols = [col for col in df_clean.columns if col.startswith("is_")]

for col in binary_cols:
    unique_vals = df_clean[col].unique()
    df_clean[col] = df_clean[col].map(
        {"Yes": 1, "No": 0, "True": 1, "False": 0, "1": 1, "0": 0}
    )
    df_clean[col] = df_clean[col].astype(int)

print(f"-> {len(binary_cols)} binary columns converted to 0/1.")
df_clean.head()

-> 17 binary columns converted to 0/1.


,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,9.3,1.2,41,C8,8794,C2,M4,Diesel,250Nm@2750rpm,113.45bhp@4000rpm,...,1,1,1,1,1,0,1,1,3,0
1,8.2,1.8,35,C2,27003,C1,M9,Diesel,200Nm@1750rpm,97.89bhp@3600rpm,...,0,1,1,1,1,1,1,1,4,0
2,9.5,0.2,44,C8,8794,C2,M4,Diesel,250Nm@2750rpm,113.45bhp@4000rpm,...,1,1,1,1,1,0,1,1,3,0
3,5.2,0.4,44,C10,73430,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,...,0,0,0,1,0,0,0,1,0,0
4,10.1,1.0,56,C13,5410,B2,M5,Diesel,200Nm@3000rpm,88.77bhp@4000rpm,...,0,1,1,1,0,0,1,1,5,0


In [9]:
# Function to split engine specs into numeric columns
def split_engine_specs(val):
    if pd.isna(val):
        return 0, 0
    matches = re.findall(r"(\d+\.?\d*)", str(val))

    if len(matches) >= 2:
        return float(matches[0]), float(matches[1])
    elif len(matches) == 1:
        return float(matches[0]), 0
    else:
        return 0, 0


# apply to max_torque
df_clean[["torque_nm", "torque_rpm"]] = df_clean["max_torque"].apply(
    lambda x: pd.Series(split_engine_specs(x))
)

# drop max_torque after extraction
if "max_torque" in df_clean.columns:
    df_clean = df_clean.drop("max_torque", axis=1)

# apply to max_power
df_clean[["power_bhp", "power_rpm"]] = df_clean["max_power"].apply(
    lambda x: pd.Series(split_engine_specs(x))
)

# drop max_power after extraction
if "max_power" in df_clean.columns:
    df_clean = df_clean.drop("max_power", axis=1)

print("Created new numeric columns: torque_nm, torque_rpm, power_bhp, power_rpm")
df_clean[["torque_nm", "torque_rpm", "power_bhp", "power_rpm"]].head()

Created new numeric columns: torque_nm, torque_rpm, power_bhp, power_rpm


,torque_nm,torque_rpm,power_bhp,power_rpm
0,250.0,2750.0,113.45,4000.0
1,200.0,1750.0,97.89,3600.0
2,250.0,2750.0,113.45,4000.0
3,60.0,3500.0,40.36,6000.0
4,200.0,3000.0,88.77,4000.0


In [10]:
# check redundancy between region_code and region_density
redundancy_check = df_clean.groupby("region_code")["region_density"].nunique()
if (
    redundancy_check.max() == 1
    and df_clean["region_code"].nunique() == df_clean["region_density"].nunique()
):
    print("Region_code and Region_Density are 100% redundant.")
    print("-> each region_code maps to a single region_density value.")

    df_clean = df_clean.drop("region_code", axis=1)
else:
    print("region_code and region_density are not fully redundant.")
    print(f"-> Maximum densities per code: {redundancy_check.max()}")

df_clean.head()

Region_code and Region_Density are 100% redundant.
-> each region_code maps to a single region_density value.


,subscription_length,vehicle_age,customer_age,region_density,segment,model,fuel_type,engine_type,airbags,is_esc,...,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status,torque_nm,torque_rpm,power_bhp,power_rpm
0,9.3,1.2,41,8794,C2,M4,Diesel,1.5 L U2 CRDi,6,1,...,1,0,1,1,3,0,250.0,2750.0,113.45,4000.0
1,8.2,1.8,35,27003,C1,M9,Diesel,i-DTEC,2,0,...,1,1,1,1,4,0,200.0,1750.0,97.89,3600.0
2,9.5,0.2,44,8794,C2,M4,Diesel,1.5 L U2 CRDi,6,1,...,1,0,1,1,3,0,250.0,2750.0,113.45,4000.0
3,5.2,0.4,44,73430,A,M1,CNG,F8D Petrol Engine,2,0,...,0,0,0,1,0,0,60.0,3500.0,40.36,6000.0
4,10.1,1.0,56,5410,B2,M5,Diesel,1.5 Turbocharged Revotorq,2,0,...,0,0,1,1,5,0,200.0,3000.0,88.77,4000.0


In [11]:
# Cleaning categorical columns by stripping whitespace
cat_cols = df_clean.select_dtypes(include=["str"]).columns

for col in cat_cols:
    df_clean[col] = df_clean[col].str.strip()
    print(f"-> {col} cleaned.")
df_clean.head()

-> segment cleaned.
-> model cleaned.
-> fuel_type cleaned.
-> engine_type cleaned.
-> rear_brakes_type cleaned.
-> transmission_type cleaned.
-> steering_type cleaned.


,subscription_length,vehicle_age,customer_age,region_density,segment,model,fuel_type,engine_type,airbags,is_esc,...,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status,torque_nm,torque_rpm,power_bhp,power_rpm
0,9.3,1.2,41,8794,C2,M4,Diesel,1.5 L U2 CRDi,6,1,...,1,0,1,1,3,0,250.0,2750.0,113.45,4000.0
1,8.2,1.8,35,27003,C1,M9,Diesel,i-DTEC,2,0,...,1,1,1,1,4,0,200.0,1750.0,97.89,3600.0
2,9.5,0.2,44,8794,C2,M4,Diesel,1.5 L U2 CRDi,6,1,...,1,0,1,1,3,0,250.0,2750.0,113.45,4000.0
3,5.2,0.4,44,73430,A,M1,CNG,F8D Petrol Engine,2,0,...,0,0,0,1,0,0,60.0,3500.0,40.36,6000.0
4,10.1,1.0,56,5410,B2,M5,Diesel,1.5 Turbocharged Revotorq,2,0,...,0,0,1,1,5,0,200.0,3000.0,88.77,4000.0


In [12]:
data_health_check(df_clean)

1. Dimensiones del Dataset: (58592, 41)

2. Duplicados:
   Filas duplicadas: 1894

3. Tipos de Datos y Valores Nulos:


,Dtype,Nunique,Missing_Values,%_Missing
subscription_length,float64,140,0,0.0
vehicle_age,float64,49,0,0.0
customer_age,int64,41,0,0.0
region_density,int64,22,0,0.0
segment,str,6,0,0.0
model,str,11,0,0.0
fuel_type,str,3,0,0.0
engine_type,str,11,0,0.0
airbags,int64,3,0,0.0
is_esc,int64,2,0,0.0


In [13]:
# 1. Verify exact duplicates (Everything same, incl. target)
duplicados_exactos = df_clean.duplicated().sum()

# 2. Verify conflicting duplicates (Same features, different target)
cols_features = [c for c in df_clean.columns if c != "claim_status"]
conflictos = df_clean.groupby(cols_features)["claim_status"].nunique()
# If nunique > 1, it means that for those features there are 0 and 1 at the same time
n_conflictos = conflictos[conflictos > 1].shape[0]

print("Duplicate Analysis:")
print(f"-> Totally identical records: {duplicados_exactos}")
print(f"-> Identical profiles with different outcomes (Conflicts): {n_conflictos}")

# 3. ACTION: REMOVE DUPLICATES
# Keep the first occurrence and delete the rest
df_clean = df_clean.drop_duplicates(keep="first")

print("\nDuplicates have been removed.")
print(f"Final dimensions of the dataset: {df_clean.shape}")

Duplicate Analysis:
-> Totally identical records: 1894
-> Identical profiles with different outcomes (Conflicts): 262

Duplicates have been removed.
Final dimensions of the dataset: (56698, 41)


## 2.2 Estrategia de Particionamiento (Data Splitting)

Una vez garantizada la integridad estructural de los datos, procederemos a la división estratégica del dataset. Este paso se realiza **antes** del análisis exploratorio profundo (Deep EDA) para cumplir con el principio de **"Ceguera del Evaluador"** y evitar la fuga de información (*Data Leakage*).

La estrategia consiste en:

1.  **Segregación del Test Set (Hold-out):** Se separará un **20%** de los datos como conjunto de prueba final. Este subconjunto permanecerá aislado y no será visualizado ni utilizado para tomar decisiones estadísticas (como imputación de nulos o tratamiento de outliers) durante la fase de exploración.
2.  **Estratificación:** Dado el desbalance de la variable objetivo (6.4% de reclamos), el particionamiento será **estratificado**. Esto garantiza que tanto el conjunto de entrenamiento como el de prueba mantengan la misma proporción de siniestralidad que la población original, asegurando una evaluación justa.
3.  **Base para EDA:** El Análisis Exploratorio de Datos subsiguiente se ejecutará exclusivamente sobre el **80% de Entrenamiento**, permitiendo descubrir patrones generalizables sin contaminar nuestra visión con los datos de prueba.

In [14]:
# 1. split features and target
X = df_clean.drop(["claim_status"], axis=1)
y = df_clean["claim_status"]

# 2. Create the Final Test Set (20%)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# 3. Prepare the Full Training DataFrame
train_df = pd.concat([X_train_full, y_train_full], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Reset index for both sets
train_df = train_df.reset_index(drop=True)

# 4. Create Stratified K-Folds in the Training Set
# Initialize the column to -1
train_df["kfold"] = -1

# Initialize StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Fill the 'kfold' column
for fold, (_train_idx, val_idx) in enumerate(
    kf.split(X=train_df, y=train_df["claim_status"])
):
    train_df.loc[val_idx, "kfold"] = fold

# 5. Summary of the splits
print("General Dimensions:")
print(f"-> Train Set (for CV): {train_df.shape[0]} rows (80%)")
print(f"-> Test Set (Hold-out): {test_df.shape[0]} rows (20%)")

print("\nDistribution of Folds in Train Set:")
print(train_df.kfold.value_counts())

# 6. Save the files
PROCESSED_DATA_PATH.mkdir(parents=True, exist_ok=True)

# Save the train set with the folds column and the clean test set
train_df.to_csv(PROCESSED_DATA_PATH / "train_folds.csv", index=False)
test_df.to_csv(PROCESSED_DATA_PATH / "test.csv", index=False)

General Dimensions:
-> Train Set (for CV): 45358 rows (80%)
-> Test Set (Hold-out): 11340 rows (20%)

Distribution of Folds in Train Set:
kfold
2    9072
1    9072
0    9072
3    9071
4    9071
Name: count, dtype: int64
